In [1]:
# Imports
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta

import geopandas as gpd
from geopy import distance
from shapely.geometry import Point, Polygon, LineString
from shapely import wkt
import math
from math import sqrt, pow, atan2, degrees, pi
from shapely import affinity

import subprocess



from haversine import haversine, Unit

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Constants
data_path = "../../data/"
taxi_raw = data_path + "Taxi_Trips.csv"
rideshare_raw = data_path + "Transportation_Network_Providers_-_Trips.csv"
census_raw = "Boundaries - Census Tracts - 2010.geojson"

In [3]:
#read data
taxi = pd.read_csv(taxi_raw)
rideshare = pd.read_csv(rideshare_raw)
census = gpd.read_file(census_raw)

In [27]:
taxi.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,fc1253c0ecd73cb81c09e26a695cb16b6c2bee2d,f99d866ed2543d082bef92509e7e1f435207b128304932...,10/01/2019 12:00:00 AM,10/01/2019 12:15:00 AM,673.0,2.80,NaN,NaN,3.0,1.0,...,0.0,10.00,Cash,City Service,41.965812,-87.655879,POINT (-87.65588 41.96581),42.009623,-87.670167,POINT (-87.670 42.010)
1,f82f0fe1170708e9d66b113a7b6215d4617f8a0c,0639daf7ae96a512de2bfe868e999eeb0fb34bdca74610...,10/01/2019 12:00:00 AM,10/01/2019 12:30:00 AM,1621.0,13.08,NaN,NaN,32.0,NaN,...,0.0,34.25,Cash,Flash Cab,41.878866,-87.625192,POINT (-87.62519 41.87887),NaN,NaN,POINT (nan nan)
2,f207a4e0e6d1473ce56ddf89e00025c0b753fd23,19405f5540015239a3c20694968bdf00929f0afccc624d...,10/01/2019 12:00:00 AM,10/01/2019 12:15:00 AM,660.0,4.04,NaN,NaN,32.0,24.0,...,0.0,13.00,Cash,Patriot Taxi Dba Peace Taxi Associat,41.878866,-87.625192,POINT (-87.62519 41.87887),41.901207,-87.676356,POINT (-87.676 41.901)
3,f1488983573a9c99bb9edef3feeb92871e22ef5c,7e62180fd8402ef87fb6d789a65bd64251e62d34f66b99...,10/01/2019 12:00:00 AM,10/01/2019 12:15:00 AM,840.0,0.50,NaN,NaN,28.0,42.0,...,0.0,24.00,Cash,Taxi Affiliation Services,41.874005,-87.663518,POINT (-87.66352 41.87401),41.778877,-87.594925,POINT (-87.595 41.779)
4,ef45440690a5c4557a5738a68174b9460a42fdde,171ec426eaf8f54c5acbb7e3fde8e0683bfa6042af0b00...,10/01/2019 12:00:00 AM,10/01/2019 12:15:00 AM,360.0,0.90,1.703108e+10,1.703108e+10,8.0,8.0,...,1.0,6.75,Cash,Taxi Affiliation Services,41.898332,-87.620763,POINT (-87.62076 41.89833),41.892508,-87.626215,POINT (-87.626 41.893)


In [29]:
#converting the taxi data and rideshare data into geopandas dfs
taxi['Pickup Centroid Location'] = gpd.points_from_xy(taxi["Pickup Centroid Longitude"], taxi["Pickup Centroid Latitude"])
taxi['Dropoff Centroid Location'] = gpd.points_from_xy(taxi['Dropoff Centroid Longitude'], taxi['Dropoff Centroid Latitude'])
taxi_pickup = gpd.GeoDataFrame(taxi, geometry='Pickup Centroid Location')
taxi_dropoff = gpd.GeoDataFrame(taxi, geometry='Dropoff Centroid  Location')

rideshare['Pickup Centroid Location'] = gpd.points_from_xy(rideshare["Pickup Centroid Longitude"], rideshare["Pickup Centroid Latitude"])
rideshare['Dropoff Centroid Location'] = gpd.points_from_xy(rideshare['Dropoff Centroid Longitude'], rideshare['Dropoff Centroid Latitude'])
rideshare_pickup = gpd.GeoDataFrame(rideshare, geometry='Pickup Centroid Location')
rideshare_dropoff = gpd.GeoDataFrame(rideshare, geometry='Dropoff Centroid Location')


In [30]:
print(taxi.shape)
print(taxi_pickup.shape)
print(taxi_dropoff.shape)
print(rideshare.shape)
print(rideshare_pickup.shape)
print(rideshare_dropoff.shape)

(1449712, 24)
(1449712, 24)
(1449712, 24)
(9503882, 21)
(9503882, 21)
(9503882, 21)


In [31]:
#census.reset_index(inplace=True, drop=True)
#taxi_pickup.reset_index(inplace=True, drop=True)

census.crs = taxi_pickup.crs
taxi_pickup_merged = gpd.sjoin(taxi_pickup, census,  op = 'intersects', how= 'right')
taxi_dropoff_merged = gpd.sjoin(taxi_dropoff, census,  op = 'intersects', how= 'right')

rideshare_pickup_merged = gpd.sjoin(rideshare_pickup, census,  op = 'intersects', how= 'right')
rideshare_dropoff_merged = gpd.sjoin(rideshare_dropoff, census,  op = 'intersects', how= 'right')



/Users/greeshamsimon/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
/Users/greeshamsimon/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
/Users/greeshamsimon/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
/Users/greeshamsimon/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [33]:
rideshare_pickup_summary = pd.DataFrame(rideshare_pickup_merged.name10.value_counts()).reset_index()
rideshare_pickup_summary.rename(columns={"index": "name10", "name10": "rideshare_pickup"}, inplace = True)
print(rideshare_pickup_summary.shape)
rideshare_dropoff_summary = pd.DataFrame(rideshare_dropoff_merged.name10.value_counts()).reset_index()
rideshare_dropoff_summary.rename(columns={"index": "name10", "name10": "rideshare_dropoff"}, inplace = True)
print(rideshare_dropoff_summary.shape)
taxi_pickup_summary = pd.DataFrame(taxi_pickup_merged.name10.value_counts()).reset_index()
taxi_pickup_summary.rename(columns={"index": "name10", "name10": "taxi_pickup"}, inplace = True)
print(taxi_pickup_summary.shape)
taxi_dropoff_summary = pd.DataFrame(taxi_dropoff_merged.name10.value_counts()).reset_index()
taxi_dropoff_summary.rename(columns={"index": "name10", "name10": "taxi_dropoff"}, inplace = True)
print(taxi_dropoff_summary.shape)

(801, 2)
(801, 2)
(801, 2)
(801, 2)


In [34]:
heatmap_values = taxi_dropoff_summary.merge(taxi_pickup_summary, on = 'name10').merge(rideshare_pickup_summary, on = 'name10').merge(rideshare_dropoff_summary, on = 'name10')

In [35]:
heatmap = census.merge(heatmap_values, on = 'name10')

In [142]:
heatmap["taxi_pickup_percent"] = heatmap["taxi_pickup"]/sum(heatmap["taxi_pickup"])*100
heatmap["taxi_dropoff_percent"] = heatmap["taxi_dropoff"]/sum(heatmap["taxi_dropoff"])*100
heatmap["rideshare_pickup_percent"] = heatmap["rideshare_pickup"]/sum(heatmap["rideshare_pickup"])*100
heatmap["rideshare_dropoff_percent"] = heatmap["rideshare_dropoff"]/sum(heatmap["rideshare_dropoff"])*100
heatmap

,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10,geometry,taxi_dropoff,taxi_pickup,rideshare_pickup,rideshare_dropoff,taxi_pickup_percent,taxi_dropoff_percent,rideshare_pickup_percent,rideshare_dropoff_percent
0,17,8424,44,Census Tract 8424,44,17031842400,,842400,031,"MULTIPOLYGON (((-87.62405 41.73022, -87.62405 ...",14,3,8028,8417,0.000223,0.001071,0.090575,0.095708
1,17,8403,59,Census Tract 8403,59,17031840300,,840300,031,"MULTIPOLYGON (((-87.68608 41.82296, -87.68607 ...",1251,1289,1304,1248,0.095964,0.095694,0.014712,0.014191
2,17,8411,34,Census Tract 8411,34,17031841100,,841100,031,"MULTIPOLYGON (((-87.62935 41.85280, -87.62934 ...",1218,87,28309,26513,0.006477,0.093170,0.319392,0.301475
3,17,8412,31,Census Tract 8412,31,17031841200,,841200,031,"MULTIPOLYGON (((-87.68813 41.85569, -87.68816 ...",1,1,7066,6067,0.000074,0.000076,0.079721,0.068987
4,17,8390,32,Census Tract 8390,32,17031839000,,839000,031,"MULTIPOLYGON (((-87.63312 41.87449, -87.63306 ...",6166,711,61910,52112,0.052933,0.471661,0.698491,0.592557
5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031,"MULTIPOLYGON (((-87.66782 41.87418, -87.66768 ...",2763,1450,29026,33559,0.107951,0.211353,0.327482,0.381594
6,17,6503.01,65,Census Tract 6503.01,65,17031650301,,650301,031,"MULTIPOLYGON (((-87.73706 41.77120, -87.73718 ...",1,1,875,530,0.000074,0.000076,0.009872,0.006027
7,17,5305.03,53,Census Tract 5305.03,53,17031530503,,530503,031,"MULTIPOLYGON (((-87.64386 41.66321, -87.64362 ...",1,1,605,476,0.000074,0.000076,0.006826,0.005413
8,17,7608.03,76,Census Tract 7608.03,76,17031760803,,760803,031,"MULTIPOLYGON (((-87.83844 41.97020, -87.83853 ...",1,1,1013,910,0.000074,0.000076,0.011429,0.010347
9,17,306.01,77,Census Tract 306.01,77,17031030601,,030601,031,"MULTIPOLYGON (((-87.65438 41.99020, -87.64893 ...",24,9,3304,3586,0.000670,0.001836,0.037277,0.040776


In [143]:
heatmap.to_file("heatmap.geojson", driver='GeoJSON')

## Aggregating Fare for Taxed Census Tracts

In [50]:
downtown = ["8423","8422","804","803","802.01","801","812.02","802.02",
          "8383","810","811","812.01","813","818","819","814.01","817",
            "815","818","816","814.03","3201","816","814.03","3201","816",
           "2435","2434","2433","8330","8331","2801","2819","8391","3204",
           "3201","814.03","814.02","8419","8390","3206"]

In [53]:
taxi_pickup_tax = taxi_pickup_merged[taxi_pickup_merged.name10.isin(downtown)]
taxi_dropoff_tax = taxi_dropoff_merged[taxi_dropoff_merged.name10.isin(downtown)]
rideshare_pickup_tax = rideshare_pickup_merged[rideshare_pickup_merged.name10.isin(downtown)]
rideshare_dropoff_tax = rideshare_dropoff_merged[rideshare_dropoff_merged.name10.isin(downtown)]


In [67]:
taxi_pickup_tax["relevantTimeStamp"] = pd.to_datetime(taxi_pickup_tax["Trip Start Timestamp"])
rideshare_pickup_tax["relevantTimeStamp"] = pd.to_datetime(rideshare_pickup_tax["Trip Start Timestamp"])
taxi_dropoff_tax["relevantTimeStamp"] = pd.to_datetime(taxi_dropoff_tax["Trip End Timestamp"])
rideshare_dropoff_tax["relevantTimeStamp"] = pd.to_datetime(rideshare_dropoff_tax["Trip End Timestamp"])

/Users/greeshamsimon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/greeshamsimon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/greeshamsimon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [79]:
#filtering rides that are monday through friday (0-4)
taxi_dropoff_tax = taxi_dropoff_tax[taxi_dropoff_tax["relevantTimeStamp"].dt.dayofweek<5]
taxi_pickup_tax = taxi_pickup_tax[taxi_pickup_tax["relevantTimeStamp"].dt.dayofweek<5]
rideshare_dropoff_tax = rideshare_dropoff_tax[rideshare_dropoff_tax["relevantTimeStamp"].dt.dayofweek<5]
rideshare_pickup_tax = rideshare_pickup_tax[rideshare_pickup_tax["relevantTimeStamp"].dt.dayofweek<5]

In [130]:
avg_pickup_taxi = pd.DataFrame(taxi_pickup_tax.groupby('name10')['Fare']
                               .mean()).rename(columns = {"Fare":"Average Taxi Pickup Fare"}).reset_index()
avg_dropoff_taxi = pd.DataFrame(taxi_dropoff_tax.groupby('name10')['Fare']
                               .mean()).rename(columns = {"Fare":"Average Taxi Dropoff Fare"}).reset_index()
avg_pickup_rideshare = pd.DataFrame(rideshare_pickup_tax.groupby('name10')['Fare']
                               .mean()).rename(columns = {"Fare":"Average Rideshare Pickup Fare"}).reset_index()
avg_dropoff_rideshare = pd.DataFrame(rideshare_dropoff_tax.groupby('name10')['Fare']
                               .mean()).rename(columns = {"Fare":"Average Rideshare Dropoff Fare"}).reset_index()

heatmap_downtown_fares = avg_dropoff_taxi.merge(avg_pickup_taxi, how= "left").merge(avg_pickup_rideshare).merge(avg_dropoff_rideshare)

In [131]:
heatmap_downtown_fares1 = census.merge(heatmap_downtown_fares, on = 'name10', how = "inner")
heatmap_downtown_fares2 = census.merge(heatmap_downtown_fares, on = 'name10', how = "left")

In [132]:
gpd.GeoDataFrame(heatmap_downtown_fares1).to_file("fare_heatmap_downtown.geojson", driver='GeoJSON')
gpd.GeoDataFrame(heatmap_downtown_fares2).to_file("fare_heatmap_chicagoAll.geojson", driver='GeoJSON')

In [136]:
heatmap_downtown_fares.mean()

Average Taxi Dropoff Fare         11.593551
Average Taxi Pickup Fare          10.142991
Average Rideshare Pickup Fare      9.811092
Average Rideshare Dropoff Fare     9.466551
dtype: float64

In [3]:
test = gpd.read_file('fare_heatmap_chicagoAll.geojson')

In [5]:
downtown = ["8423","8422","804","803","802.01","801","812.02","802.02",
          "8383","810","811","812.01","813","818","819","814.01","817",
            "815","818","816","814.03","3201","816","814.03","3201","816",
           "2435","2434","2433","8330","8331","2801","2819","8391","3204",
           "3201","814.03","814.02","8419","8390","3206"]
test2 = test[test.name10.isin(downtown)]


In [7]:
test2.to_file("fare_heatmap_downtown2.geojson", driver='GeoJSON')

## Miscellaneous

In [9]:
census.head()
census.shape

(801, 10)

In [3]:
heatmap = gpd.read_file("heatmap.geojson")
heatmap.shape

(801, 18)

In [42]:
heatmap_csv = pd.DataFrame(heatmap)

In [6]:
data = gpd.read_file('fare_heatmap_chicagoAll.geojson')

In [8]:
data2 = gpd.read_file('fare_heatmap_downtown.geojson')

In [29]:
list(data2.geometry[data2['name10']=='8390']) == list(data.geometry[data['name10']=='8390'])

True

In [26]:
data.geometry[data['name10']=='8390']

,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10,Average Taxi Dropoff Fare,Average Taxi Pickup Fare,Average Rideshare Pickup Fare,Average Rideshare Dropoff Fare,geometry
0,17,8424,44,Census Tract 8424,44,17031842400,,842400,031,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-87.62405 41.73022, -87.62405 ..."
1,17,8403,59,Census Tract 8403,59,17031840300,,840300,031,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-87.68608 41.82296, -87.68607 ..."
2,17,8411,34,Census Tract 8411,34,17031841100,,841100,031,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-87.62935 41.85280, -87.62934 ..."
3,17,8412,31,Census Tract 8412,31,17031841200,,841200,031,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-87.68813 41.85569, -87.68816 ..."
4,17,8390,32,Census Tract 8390,32,17031839000,,839000,031,10.558113,17.172812,8.845057,8.889168,"MULTIPOLYGON (((-87.63312 41.87449, -87.63306 ..."
5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-87.66782 41.87418, -87.66768 ..."
6,17,6503.01,65,Census Tract 6503.01,65,17031650301,,650301,031,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-87.73706 41.77120, -87.73718 ..."
7,17,5305.03,53,Census Tract 5305.03,53,17031530503,,530503,031,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-87.64386 41.66321, -87.64362 ..."
8,17,7608.03,76,Census Tract 7608.03,76,17031760803,,760803,031,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-87.83844 41.97020, -87.83853 ..."
9,17,306.01,77,Census Tract 306.01,77,17031030601,,030601,031,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-87.65438 41.99020, -87.64893 ..."
